In [1]:
import numpy as np
import itertools
from scipy.integrate import odeint
from sklearn.preprocessing import StandardScaler
from models.CR import CR3D
from models.ESN import ESN3D
from models.SparseESN import SparseESN3D
from metrics.metrics import compute_valid_prediction_time

In [2]:
def lorenz_deriv(state, t, sigma=10.0, rho=28.0, beta=8.0/3.0):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x*(rho - z) - y
    dzdt = x*y - beta*z
    return [dxdt, dydt, dzdt]

def generate_lorenz_data(
    initial_state=[1.0, 1.0, 1.0],
    tmax=25.0,
    dt=0.01,
    sigma=10.0,
    rho=28.0,
    beta=8.0/3.0
):
    num_steps = int(tmax / dt) + 1 # +1 to include t=0
    t_vals = np.linspace(0, tmax, num_steps)
    sol = odeint(lorenz_deriv, initial_state, t_vals, args=(sigma, rho, beta))
    return t_vals, sol

def report_vpt(name, preds, test_target, time_test, dt):
    T_VPT, T_lambda, ratio = compute_valid_prediction_time(
        test_target, preds, time_test, threshold=0.4, lambda_max=0.9, dt=dt
    )
    # print(f"{name:20s} => T_VPT={T_VPT:.3f},  T_lambda={T_lambda:.3f}, ratio={ratio:.3f}")
    return T_VPT, T_lambda, ratio

# 1) Generate Lorenz data
tmax = 110
dt   = 0.01
t_vals, lorenz_traj = generate_lorenz_data(
    initial_state=[1.0,1.0,1.0],
    tmax=tmax,
    dt=dt
)

washout = 1000
t_vals = t_vals[washout:]
lorenz_traj = lorenz_traj[washout:]

# normalize
scaler = StandardScaler()
scaler.fit(lorenz_traj)
lorenz_traj = scaler.transform(lorenz_traj)

T_data = len(lorenz_traj)
print(f"Data length: {T_data}, from t=0..{tmax} with dt={dt}.")

n_test_steps = 3000

# train/test split
train_frac = 0.7
train_end = int(train_frac*(T_data-1))
train_input  = lorenz_traj[:train_end]
train_target = lorenz_traj[1:train_end+1]
test_input   = lorenz_traj[train_end:train_end+n_test_steps]
test_target  = lorenz_traj[train_end+1:train_end+n_test_steps+1]
print(f"Train size: {len(train_input)}  Test size: {len(test_input)}")

initial_in = test_input[0]

Data length: 10001, from t=0..110 with dt=0.01.
Train size: 7000  Test size: 3000


In [3]:
# (a) Baseline ESN
esn = ESN3D(
    reservoir_size=300,
    spectral_radius=0.95,
    input_scale=1.0,
    leaking_rate=0.8,
    ridge_alpha=1e-6,
    seed=42
)
esn.fit_readout(train_input, train_target, discard=100)

# (b) Cycle Reservoir
cycle_res = CR3D(
    reservoir_size=300,
    spectral_radius=0.95,
    input_scale=1.0,
    leaking_rate=0.8,
    ridge_alpha=1e-6,
    seed=43
)
cycle_res.fit_readout(train_input, train_target, discard=100)

# (c) Sparse ESN
sparse_res = SparseESN3D(
    reservoir_size=300,
    spectral_radius=0.95,
    connectivity=0.04,
    input_scale=1.0,
    leaking_rate=0.8,
    ridge_alpha=1e-6,
    seed=44
)
sparse_res.fit_readout(train_input, train_target, discard=100)

esn_preds    = esn.predict_autoregressive(initial_in, n_test_steps)
cycle_preds  = cycle_res.predict_autoregressive(initial_in, n_test_steps)
sparse_preds = sparse_res.predict_autoregressive(initial_in, n_test_steps)

report_vpt("Dense ESN",    esn_preds, test_target, t_vals[train_end:train_end+n_test_steps], dt)
report_vpt("Cycle Res",  cycle_preds, test_target, t_vals[train_end:train_end+n_test_steps], dt)
report_vpt("Sparse ESN",   sparse_preds, test_target, t_vals[train_end:train_end+n_test_steps], dt)

(121.99999999999989, 1.1111111111111112, 73.098)

In [4]:
# Define your parameter grids
param_grid_common = {
    "spectral_radius": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2],
    "input_scale": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    "leaking_rate": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    "ridge_alpha": [1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9]
}

param_grid_sparse = {
    **param_grid_common,
    "connectivity": [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]
}

def run_grid_search_top2(model_class, param_grid, model_name):
    scored_params = []
    all_combinations = list(itertools.product(*param_grid.values()))

    for comb in all_combinations:
        params = dict(zip(param_grid.keys(), comb))
        model = model_class(reservoir_size=300, **params)
        model.fit_readout(train_input, train_target, discard=100)
        preds = model.predict_autoregressive(initial_in, n_test_steps)
        T_VPT, _, _ = compute_valid_prediction_time(
            test_target, preds, t_vals[train_end:train_end+n_test_steps],
            threshold=0.4, lambda_max=0.9, dt=dt
        )
        scored_params.append((T_VPT, params))

    # Sort by T_VPT descending and pick top 2
    top_2 = sorted(scored_params, key=lambda x: x[0], reverse=True)[:2]

    print(f"Top 2 results for {model_name}:")
    for i, (score, params) in enumerate(top_2):
        print(f"{i+1}) T_VPT={score:.3f} with params {params}")

    return top_2

In [5]:
top2_esn     = run_grid_search_top2(ESN3D, param_grid_common, "Dense ESN")
top2_cr      = run_grid_search_top2(CR3D, param_grid_common, "Cycle Res")
top2_sparse  = run_grid_search_top2(SparseESN3D, param_grid_sparse, "Sparse ESN")

KeyboardInterrupt: 